<a href="https://colab.research.google.com/github/zanzivyr/Optimizers/blob/main/LP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linear Programming

Today we will continue our study of optimization with a brief overview of Linear Programming.

## Resources

These are resources I used to learn about Linear Programming:

- The Organic Chemistry Tutor - https://youtu.be/Bzzqx1F23a8
- Meghan de Witt - https://youtu.be/rzRZLGD_aeE
- Wikipedia - https://en.wikipedia.org/wiki/Linear_programming
- Timeparticle - https://www.youtube.com/watch?v=5bxsxM2UTb4
- Geeks for Geeks - https://www.geeksforgeeks.org/singular-value-decomposition-svd/
- Section.io - https://www.section.io/engineering-education/moore-penrose-pseudoinverse/
- Jon Krohn - https://youtu.be/vXk-o3PVUdU

In [16]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

## Instantiate

First we'll create a LP with an objective function and 2 constraint inequalities as a matrix.

We can use random integer values for the coefficients.

The A matrix shall be constructed as:

1. **Row 1** = coefficients of the **objective**
2. **Row 2+** = coefficients of the **constraints**
3. **Column vector b** = solutions to the objective and constraints

### Augmentations

We're also going to augment the objective and constraints.

By adding "slack" variables to the constraints (one per constraint) we can convert the inequality to an equality. (Basically, by having another term, we can guarantee that the equation will become equal... it's a stop gap) Then when checking that the solutions to the variables meet the final constraint, we can say if our solution is feasible, or unfeasible.

**Note**: The objective row should have zeroes for the slack variables.

This is called the Simplex Method

In [17]:
A = tf.random.normal(
    shape=[3,2],
    mean=0,
    stddev=1,
    dtype=tf.dtypes.float32,
    seed=42,
    name=None
)
eye = tf.constant([[0,0],[1,0],[0,1]], dtype=tf.dtypes.float32)
b = tf.random.normal(
    shape=[3,1],
    mean=0,
    stddev=1,
    dtype=tf.dtypes.float32,
    seed=42,
    name=None
)
A = tf.concat([A, eye], 1)
A, b

(<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
 array([[ 1.2180098 , -0.39854664,  0.        ,  0.        ],
        [ 0.63967335, -0.63211644,  1.        ,  0.        ],
        [-1.8190719 , -0.9362425 ,  0.        ,  1.        ]],
       dtype=float32)>, <tf.Tensor: shape=(3, 1), dtype=float32, numpy=
 array([[ 0.23851669],
        [-0.7702351 ],
        [-0.99450314]], dtype=float32)>)

## Singular Value Decomposition (SVD)

Use the Singular Value Decomposition to find the Moore-Penrose Pseudoinverse

In [18]:
# SVD
u, s, vt = np.linalg.svd(A, full_matrices=True)
u, s, vt

(array([[-0.40928856, -0.39557117, -0.82219607],
        [-0.21294492, -0.83483064,  0.5076537 ],
        [ 0.8872076 , -0.38285932, -0.25745174]], dtype=float32),
 array([2.484513 , 1.4265596, 0.6548556], dtype=float32),
 array([[-0.9050575 , -0.21449472, -0.08570892,  0.35709518],
        [-0.2238812 ,  0.73169947, -0.58520555, -0.26837948],
        [-0.31821835,  0.37844145,  0.77521473, -0.3931428 ],
        [ 0.1717127 ,  0.5247761 ,  0.22187957,  0.8036755 ]],
       dtype=float32))

### SVD Inverse

Using the SVD, we can now invert the matrix

In [19]:
# Moore-Penrose Pseudo Inverse
vtt = np.transpose(vt)
s = np.linalg.inv(np.diag(s))
ut = np.transpose(u)

vtt.shape, s.shape, ut.shape

((4, 4), (3, 3), (3, 3))

Σ transpose needs to be the same dimensions as A.

We can concatenate on another column if A is overdetermined or row if A is underdetermined.

In [20]:
# Overdetermined
# Concat extra column of zeroes
if A.shape[0] > A.shape[1]:
  print("Overdetermined")
  sinv = np.concatenate(
      (
          s, 
          tf.zeros([s.shape[0], 1], tf.float32)
      ), 
      axis=1
  )
# Underdetermined
# Concat extra row of zeroes
elif A.shape[0] < A.shape[1]:
  print("Underdetermined")
  sinv = np.concatenate(
      (
          s, 
          tf.zeros([1, s.shape[1]], tf.float32)
      ), 
      axis=0
  )
else:
  sinv = s

sinv.shape

Underdetermined


(4, 3)

## Moore-Penrose Pseudoinverse

Now we can find A dagger:

A† = VTᵀ Σ Uᵀ

In [21]:
Adag = np.dot(vtt, np.dot(sinv, ut))
Adag

array([[ 0.6107108 , -0.03809952, -0.13800131],
       [-0.64270586, -0.11643712, -0.42174965],
       [-0.79692036,  0.9507695 , -0.17831913],
       [ 0.50919837, -0.17831914,  0.35410577]], dtype=float32)

A† is the pseudoinverse. 

Now we can solve the linear equation for x:

AA† x = A† b

In [22]:
x = np.dot(Adag, b)
x

array([[ 0.31225306],
       [ 0.35581926],
       [-0.7450559 ],
       [-0.09335932]], dtype=float32)

## Optimizing the objective

Since we have values for x1 and x2, we can now substitute these values into the objective and receive the optimized cost.

In [23]:
cost = np.dot(A[0], x)[0]

print("Cost: " + str(cost) + ", when")
i = 1
for v in x:
  print("x" + str(i) + " = " + str(v[0]))
  i+=1

Cost: 0.23851672, when
x1 = 0.31225306
x2 = 0.35581926
x3 = -0.7450559
x4 = -0.09335932


### Check against final constraints

At this point it is necessary to check against the final constraints to ensure the solution is feasible.

Usually this will look like x1, x2 ≥ 0

With the Simplex method, we also need to add the additional slack variables such that:

x1, x2, u, v, ... ≥ 0

In [24]:
flag = False
for v in range(1, len(x)):
  if(x[v] < 0):
    flag = True
    print("The solution is infeasible.")
    break

if not flag:
  print("Congrats! The solution is feasible!")

The solution is infeasible.
